In [1]:
from pathlib import Path
import os
import re
import scipy.io as sio
import numpy as np
import pandas as pd
import math

from datetime import datetime
import pynwb
from pynwb import NWBHDF5IO, NWBFile
from pynwb.file import Subject

from neuroconv.datainterfaces.ecephys.spikeglx.spikeglxdatainterface import SpikeGLXRecordingInterface
from neuroconv.tools.spikeinterface.spikeinterface import (
    add_electrodes_info_to_nwbfile, 
    add_devices_to_nwbfile, 
    add_electrode_groups_to_nwbfile,
    add_electrodes_to_nwbfile, 
    _get_electrode_table_indices_for_recording,
    _get_electrodes_table_global_ids
)
import manimoh_utils as mu
import manimoh_nwb_converters as mnc

/home/manishm/miniconda3/envs/mvdmlab_preproc_nwbconverter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Import your custom extension
from ndx_mvdmlab_metadata import (
    LabMetaDataExtension, 
    ProbeExtension, 
    OdorantInfoExtension, 
    ExperimentalBlockExtension, 
    PreprocessedAnnotationExtension
)

In [3]:
def add_lab_metadata_to_nwb(nwbfile, session_metadata):
    '''
    Function to add lab-specific metadata to the NWB file.
    '''
    # Create probe extension for probe 1
    probe1_extension = None
    if 'probe1_ID' in session_metadata:
        probe1_metadata = {}
        for key in session_metadata:
            if 'probe1_' in key and 'ID' not in key and 'location' not in key:
                if 'Depth' not in key:
                    probe1_metadata[key.split('_')[-1]] = session_metadata[key]
                else:
                    probe1_metadata['depth'] = session_metadata[key]

        probe1_extension = ProbeExtension(
            name='probe1',
            ID=session_metadata.get('probe1_ID'),
            **probe1_metadata
        )

    # Create probe extension for probe 2 (if exists)
    probe2_extension = None
    if 'probe2_ID' in session_metadata:
        probe2_metadata = {}
        for key in session_metadata:
            if 'probe2_' in key and 'ID' not in key and 'location' not in key:
                if 'Depth' not in key:
                    probe2_metadata[key.split('_')[-1]] = session_metadata[key]
                else:
                    probe2_metadata['depth'] = session_metadata[key]
        
        probe2_extension = ProbeExtension(
            name='probe2',
            ID=session_metadata.get('probe2_ID'),
            **probe2_metadata
        )

    # Create odorant info extension
    odorant_info = {}
    for odor in ['A', 'B', 'C', 'D', 'E', 'F', 'G']:
        odor_field = f'odor{odor}'
        if odor_field in session_metadata:
            key = f'Odor {odor}'
            odorant_info[key] = session_metadata[odor_field]
    
    odorant_info_extension = None
    if odorant_info:
        odorant_info_extension = OdorantInfoExtension(
            name='odorant_info',
            **odorant_info
        )

    # Create experimental block extension
    block_info = {}
    for block in [1, 2, 3]:
        block_type_field = f'block{block}_type'
        if block_type_field in session_metadata:
            block_info[block_type_field] = ','.join(session_metadata[block_type_field])
    
    block_info_extension = None
    if block_info:
        block_info_extension = ExperimentalBlockExtension(
            name='block_info',
            **block_info
        )

    # Create preprocessed annotation extension
    preprocessed_annotations = {}
    
    # Pattern for matching SWR and control channel metadata
    annotation_patterns = [
        r'imec(\d+)_shank(\d+)_SWR_channel',
        r'imec(\d+)_best_SWR_channel',
        r'imec(\d+)_best_control_channel'
    ]
    
    # Find all keys in session_metadata that match our patterns
    for key in session_metadata:
        for pattern in annotation_patterns:
            if re.match(pattern, key):
                preprocessed_annotations[key] = session_metadata[key]
                break
    
    annotation_extension = None
    if preprocessed_annotations:
        annotation_extension = PreprocessedAnnotationExtension(
            name='preprocessed_annotations',
            **preprocessed_annotations
        )

    # Prepare the metadata dictionary for LabMetaDataExtension
    lab_metadata_dict = {
        'name': 'LabMetaData',
    }
    
    # Add probes if they exist
    if probe1_extension:
        lab_metadata_dict['probe1'] = probe1_extension
    
    if probe2_extension:
        lab_metadata_dict['probe2'] = probe2_extension
    
    # Add other extensions if they exist
    if odorant_info_extension:
        lab_metadata_dict['odorant_info'] = odorant_info_extension
    
    if block_info_extension:
        lab_metadata_dict['block_info'] = block_info_extension
    
    if annotation_extension:
        lab_metadata_dict['preprocessed_annotations'] = annotation_extension

    # Populate metadata extension 
    lab_metadata = LabMetaDataExtension(**lab_metadata_dict)

    # Add to file
    nwbfile.add_lab_meta_data(lab_metadata)


In [4]:
raw_dirpath = '/mnt/datasets/incoming/mvdm/OdorSequence/sourcedata/raw/M541/M541-2024-08-31_g0'
preprocessed_dirpath = '/mnt/datasets/incoming/mvdm/OdorSequence/sourcedata/preprocessed/M541/M541-2024-08-31'
output_nwb_filepath = '/home/manishm/test_2024-08-31.nwb'
overwrite = True

In [5]:
raw_dir = Path(raw_dirpath)
preprocessed_dir = Path(preprocessed_dirpath)
output_nwb_file = Path(output_nwb_filepath)

if output_nwb_file.exists() and not overwrite:
    raise FileExistsError(f"File {output_nwb_file} already exists. Set overwrite=True to overwrite.")

# Get preprocessed_metadata present in ExpKeys file to determine which interfaces to create
preprocessed_metadata = mu.parse_expkeys(preprocessed_dir)

raw_interface = SpikeGLXRecordingInterface(folder_path=raw_dir, stream_id=f"{preprocessed_metadata['probe1_ID']}.ap")
raw_metadata = raw_interface.get_metadata()

out_nwb = NWBFile(
session_description=preprocessed_metadata['notes'], 
identifier='-'.join([preprocessed_metadata['subject'], preprocessed_metadata['date']]),
session_start_time=raw_metadata['NWBFile']['session_start_time'].astimezone(), 
experimenter=[
    preprocessed_metadata['experimenter']
],
lab="vandermeerlab",
institution="Dartmouth College",
experiment_description="Head-fixed mouse presented with odor sequences",
keywords=["ecephys", "neuropixels", "odor-sequences", "hippocampus"], # needs to be edited
)

sub_age = (datetime.strptime(preprocessed_metadata['date'], '%Y-%m-%d') - \
    datetime.strptime(preprocessed_metadata['DOB'], '%Y-%m-%d')).days//7
# Create subject
subject = Subject(subject_id=preprocessed_metadata['subject'], age=f"P{sub_age}W", 
    species = "Mus musculus", description = "Headbar-ed mouse with craniotomies over dCA1", 
    sex = preprocessed_metadata['sex'],
    )
out_nwb.subject = subject

# Add all electrodes from the first probe
# Adding depth as a property
distance_from_tip = raw_interface.recording_extractor.get_channel_locations()[:,1]
this_depth = (preprocessed_metadata["probe1_Depth"]*1000 - distance_from_tip) * \
    np.cos(math.radians(preprocessed_metadata["probe1_insertion_roll"]))
raw_interface.recording_extractor.set_property('recording_depth', values=this_depth)
# Adding label and hemisphere as additional properties
imec_id = [(x.split('.')[0]).replace('imec','') for x in raw_interface.recording_extractor.get_channel_ids()]
channel_id = [(x.split('#')[-1]) for x in raw_interface.recording_extractor.get_channel_ids()]
shank_id =  raw_interface.recording_extractor.get_channel_groups().tolist()
this_label = [f"imec{imec_id[x]}_shank{shank_id[x]}_{channel_id[x]}" for x in range(len(imec_id))]
raw_interface.recording_extractor.set_property('label', values=this_label)
this_hemi = np.repeat(preprocessed_metadata['probe1_hemisphere'], len(this_label))
raw_interface.recording_extractor.set_property('hemisphere', values=this_hemi)
# Adding description of the new column to the metadata (Otherwise nwbinspector will cry that there is no description)
raw_metadata['Ecephys']['Electrodes'].append({
    'name': 'recording_depth',
    'description': 'Depth of electrode from brain surface in micrometers, based on experimental annotation'
})
raw_metadata['Ecephys']['Electrodes'].append({
    'name': 'label',
    'description': 'Unique human-readable label for each recording channel'
})
raw_metadata['Ecephys']['Electrodes'].append({
    'name': 'hemisphere',
    'description': 'Brain hemisphere where this recording electrode was present'
})
add_electrodes_info_to_nwbfile(recording=raw_interface.recording_extractor, 
    nwbfile=out_nwb, metadata=raw_metadata)
# if 2nd probe exists, then add electrodes from that too
if 'probe2_ID' in preprocessed_metadata.keys():
    raw_interface2 = SpikeGLXRecordingInterface(folder_path=raw_dir, stream_id=f"{preprocessed_metadata['probe2_ID']}.ap")
    raw_metadata2 = raw_interface2.get_metadata()
    distance_from_tip = raw_interface2.recording_extractor.get_channel_locations()[:,1]
    this_depth = (preprocessed_metadata["probe2_Depth"]*1000 - distance_from_tip) * \
        np.cos(math.radians(preprocessed_metadata["probe2_insertion_roll"]))
    raw_interface2.recording_extractor.set_property('recording_depth', values=this_depth)
    # Adding label and hemisphere as additional properties
    imec_id2 = [(x.split('.')[0]).replace('imec','') for x in raw_interface2.recording_extractor.get_channel_ids()]
    channel_id2 = [(x.split('#')[-1]) for x in raw_interface2.recording_extractor.get_channel_ids()]
    shank_id2 =  raw_interface2.recording_extractor.get_channel_groups().tolist()
    this_label2 = [f"imec{imec_id2[x]}_shank{shank_id2[x]}_{channel_id2[x]}" for x in range(len(imec_id2))]
    raw_interface2.recording_extractor.set_property('label', values=this_label2)
    this_hemi2 = np.repeat(preprocessed_metadata['probe1_hemisphere'], len(this_label2))
    raw_interface2.recording_extractor.set_property('hemisphere', values=this_hemi2)
    # Adding description of the new column to the metadata (Otherwise nwbinspector will cry that there is no description)
    raw_metadata2['Ecephys']['Electrodes'].append({
        'name': 'recording_depth',
        'description': 'Depth of electrode from brain surface in micrometers, based on experimental annotation'
    })
    raw_metadata2['Ecephys']['Electrodes'].append({
        'name': 'label',
        'description': 'Unique human-readable label for each recording channel'
    })
    raw_metadata2['Ecephys']['Electrodes'].append({
        'name': 'hemisphere',
        'description': 'Brain hemisphere where this recording electrode was present'
    })
    add_electrodes_info_to_nwbfile(recording=raw_interface2.recording_extractor,
        nwbfile=out_nwb, metadata=raw_metadata2)

In [6]:
# Add LFP data
device_labels = []
if os.path.exists(os.path.join(preprocessed_dir,"imec0_clean_lfp.mat")):
    device_labels.append("imec0")
if os.path.exists(os.path.join(preprocessed_dir,"imec1_clean_lfp.mat")):
    device_labels.append("imec1")
# # add LFP electrodes table to nwb file
# mnc.add_lfp_electrodes_to_nwb(preprocessed_dir, out_nwb, preprocessed_metadata, device_labels)
# add LFP traces to nwb file
mnc.add_lfp_data_to_nwb(preprocessed_dir, out_nwb, preprocessed_metadata, device_labels)
    
# Add spiking data
device_labels = []
if os.path.exists(os.path.join(preprocessed_dir,"clean_units_imec0.mat")):
    device_labels.append("imec0")
if os.path.exists(os.path.join(preprocessed_dir,"clean_units_imec1.mat")):
    device_labels.append("imec1")
# add spike times, waveforms, and other information to nwb file
mnc.add_sorting_data_to_nwb(preprocessed_dir, out_nwb, preprocessed_metadata, device_labels)

# # Add behavioral epochs
# mnc.add_intervals_to_nwb(preprocessed_dir, out_nwb, preprocessed_metadata)

# # Add lab metadata using the custom extension
# mnc.add_lab_metadata_to_nwb(out_nwb, preprocessed_metadata)
    


### Write the actual NWB File

In [7]:
io = NWBHDF5IO(output_nwb_filepath, mode='w')
io.write(out_nwb)
io.close() # This is crtitcal and nwbinspector won't work without it

In [ ]:
out_nwb